# 自動走行(カメラあり)

※　ネットワーク環境によっては遅延が発生します。

TensorRTに変換したモデルを読み込みます。

In [1]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('model_trt.pth'))

model_trt_1 = TRTModule()
model_trt_1.load_state_dict(torch.load('model_trt-Copy1.pth'))


<All keys matched successfully>

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

from jetcam.csi_camera import CSICamera
camera = CSICamera(width=224, height=224, capture_fps=60)

test


widgetでCameraのOn/Offを調整できるようにします。

In [3]:
import cv2
import ipywidgets
import threading

state_widget = ipywidgets.ToggleButtons(options=['On', 'Off'], description='Camera', value='On')
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

クルマを走らせる前に、スライドバーのゲインやオフセットを準備して、走行中に調整できるようにしておきましょう。

In [4]:
import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets

network_output_slider = widgets.FloatSlider(description='推論結果', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
steering_gain_slider  = widgets.FloatSlider(description='ゲイン', min=-1.0, max=1.0, value=-0.7, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_bias_slider  = widgets.FloatSlider(description='バイアス', min=-0.5, max=0.5, value=-0.1, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_value_slider = widgets.FloatSlider(description='ハンドル値', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
throttle_slider = widgets.FloatSlider(description='速度(正:前, 負:後)', min=-1.0, max=1.0, value=0, step=0.01, orientation='vertical')


steering_gain_link   = traitlets.link((steering_gain_slider, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_bias_slider, 'value'), (car, 'steering_offset'))
#steering_value_link  = traitlets.link((steering_value_slider, 'value'), (car, 'steering'))
#throttle_slider_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))

display(
    widgets.HBox(
        [widgets.VBox([network_output_slider,
                       widgets.Label(value="X"),
                       steering_gain_slider,
                       widgets.Label(value="+"),
                       steering_bias_slider,
                       widgets.Label(value="||"), 
                       steering_value_slider], layout=Layout(
                                                    align_items='center'
                                                        )
                     ), 
         live_execution_widget,
         throttle_slider]
    )
)

最後に、以下のセルを実行すると、頂点のx値に基づいてレースカーを操り、レースカーを前進させることができます。

以下にヒントを示します。

* 車が左右にふらつく場合は、ステアリングゲインを下げてください。
* 車が曲がりきれない場合は、ステアリングゲインを上げる。
* 車が右に傾いている場合は、ステアリングバイアスをよりネガティブにする（-0.05のような小さな単位で）。
* 車が左に傾いたら、ステアリング・バイアスをよりポジティブにする（小刻みに +0.05）。

In [5]:
from utils import preprocess
from jetcam.utils import bgr8_to_jpeg

def update(change):
    global blocked_slider, robot
    new_image = change['new'] 
    
    image = preprocess(new_image).half()
    mode = 1
    if mode ==0:
        #print("Mode Switch ")
        #mode = 1
        output = model_trt_1(image).detach().cpu().numpy().flatten()
    else:
        output = model_trt(image).detach().cpu().numpy().flatten()
    
    x = float(output[0])
    #y = float(output[0])
    y = float(output[1])
    
    network_output_slider.value = x
    car.throttle = -1.0 * throttle_slider.value - 0.1 ## TT02用に修正
    #car.throttle = y*-1.0 * throttle_slider.value - 0.1 ## TT02用に修正
    steering = x * steering_gain_slider.value + steering_bias_slider.value
    if(steering<-1.0):
        steering_value_slider.value = -1.0
    elif(steering>1.0):
        steering_value_slider.value = 1.0
    else:
        steering_value_slider.value = -steering 
    car.steering = steering
    
    if(state_widget.value == 'On'):
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))  
        prediction = new_image.copy()
        prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
        
update({'new': camera.value})  # we call the function once to initialize

下記コードを実行してカメラの映像が更新されたタイミングで、update()を読み込む処理を実行します。

In [6]:
camera.observe(update, names='value') 
camera.running = True

In [7]:
camera.running = False

### データセットのタスクの定義

In [8]:
import torchvision.transforms as transforms
from xy_dataset import XYDataset

TASK = 'road_following'

CATEGORIES = ['apex']

DATASETS = ['A','donkey']
#DATASETS = ['A','donkey']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)

### データ収集(コントローラ操作用）

In [ ]:
import cv2
import ipywidgets
import threading

state_widget = ipywidgets.ToggleButtons(options=['On', 'Off'], description='Camera', value='On')
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])
import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets

network_output_slider = widgets.FloatSlider(description='推論結果', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
steering_gain_slider  = widgets.FloatSlider(description='ゲイン', min=-1.0, max=1.0, value=-0.7, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_bias_slider  = widgets.FloatSlider(description='バイアス', min=-0.5, max=0.5, value=-0.1, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_value_slider = widgets.FloatSlider(description='ハンドル値', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
throttle_slider = widgets.FloatSlider(description='速度(正:前, 負:後)', min=-1.0, max=1.0, value=0, step=0.01, orientation='vertical')


steering_gain_link   = traitlets.link((steering_gain_slider, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_bias_slider, 'value'), (car, 'steering_offset'))
#steering_value_link  = traitlets.link((steering_value_slider, 'value'), (car, 'steering'))
#throttle_slider_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))

display(
    widgets.HBox(
        [widgets.VBox([network_output_slider,
                       widgets.Label(value="X"),
                       steering_gain_slider,
                       widgets.Label(value="+"),
                       steering_bias_slider,
                       widgets.Label(value="||"), 
                       steering_value_slider], layout=Layout(
                                                    align_items='center'
                                                        )
                     ), 
         live_execution_widget,
         throttle_slider]
    )
)

from utils import preprocess
from jetcam.utils import bgr8_to_jpeg

def update(change):
    global blocked_slider, robot
    new_image = change['new'] 
    
    image = preprocess(new_image).half()
    mode = 1
    if mode ==0:
        #print("Mode Switch ")
        #mode = 1
        output = model_trt_1(image).detach().cpu().numpy().flatten()
    else:
        output = model_trt(image).detach().cpu().numpy().flatten()
    
    x = float(output[0])
    #y = float(output[0])
    y = float(output[1])
    
    network_output_slider.value = x
    car.throttle = -1.0 * throttle_slider.value - 0.1 ## TT02用に修正
    #car.throttle = y*-1.0 * throttle_slider.value - 0.1 ## TT02用に修正
    steering = x * steering_gain_slider.value + steering_bias_slider.value
    if(steering<-1.0):
        steering_value_slider.value = -1.0
    elif(steering>1.0):
        steering_value_slider.value = 1.0
    else:
        steering_value_slider.value = -steering 
    car.steering = steering
    
    if(state_widget.value == 'On'):
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))  
        prediction = new_image.copy()
        prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
        
update({'new': camera.value})  # we call the function once to initialize

camera.running = True
#~ここはデフォルトのデータ取り~~~~~~~
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget

# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        # save to disk
        dataset.save_entry(category_widget.value, camera.value, x, y)       
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = dataset.get_count(category_widget.value)
camera_widget.on_msg(save_snapshot)
data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    dataset_widget,
    category_widget,
    count_widget
])
display(data_collection_widget)

#~ここからｺﾝﾄﾛｰﾗ~~~~~~~
import time
import pygame
import math
import numpy as np
from pygame.locals import *
from Adafruit_GPIO import I2C

# pygameの初期化
pygame.init()
# ジョイスティックの初期化
pygame.joystick.init()
try:
    # ジョイスティックインスタンスの生成
    joystick = pygame.joystick.Joystick(0)
    joystick.init()
    print("Joystick Name: " + joystick.get_name())
    print("Number of Button : " + str(joystick.get_numbuttons()))
    print("Number of Axis : " + str(joystick.get_numaxes()))
    print("Number of Hats : " + str(joystick.get_numhats()))
except pygame.error:
    print('ジョイスティックが接続されていません')

def expocurve(val):
    if val >0:
        return (math.exp(val)-1)/(math.e-1)
    else:
        return (math.exp(val*-1)-1)/(math.e-1)*-1
         
# F710の操作設定
# スティック
# 左スティック#０：左右、１：上下
# 右スティック#３：左右、４：上下
axis_steer = 0
axis_accel = 4
# ボタン
# 8: 'back', 9: 'start', 10: 'Logitech',
# 0: 'A', 1: 'B', 2: 'X', 3: 'Y',
# 4: 'LB',7: 'R1',8: 'right_stick_press',
button_A = 0
button_B = 1
button_Y = 3
button_X = 2
button_S = 7

##スティックの指数関数変換
mode ="exp"

def save_snapshot_js2v(steering, throttle):
        x = steering *-1 #マシン毎要調整
        y = throttle *-1 #マシン毎要調整
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))  

        # save to disk
        dataset.save_entry(category_widget.value, camera.value, x, y)
        
        # display saved snapshot
        min, mid, max = 52, 112, 171 #出力画像値の設定、マシン毎
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot = cv2.line(snapshot, (mid, min), (mid, max), (255, 255, 255), thickness=1, lineType=cv2.LINE_4)
        snapshot = cv2.line(snapshot, (min, mid), (max, mid), (255, 255, 255), thickness=1, lineType=cv2.LINE_4)
        snapshot = cv2.rectangle(snapshot, (min, min), (max, max), (0, 255, 0)) #マシン毎要調整 52,171, (112)
        snapshot = cv2.putText(snapshot, (str(x)+":"+str(y)), (x-50, y-30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2, 8)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = dataset.get_count(category_widget.value)
        
AI = True
running = True
while running:
    for e in pygame.event.get():
        A =joystick.get_button(button_A)
        B =joystick.get_button(button_B)
        Y =joystick.get_button(button_Y)
        X =joystick.get_button(button_X)
        S =joystick.get_button(button_S)

        if A : 
            camera.unobserve_all()
            AI = False
            #camera.running = False
            print("Breaking!")
            car.steering = 0
            car.throttle = -0.5
            time.sleep(0.05)            
            car.throttle = 0
            time.sleep(0.02)            
            car.throttle = -0.5
            time.sleep(0.02)            
            car.throttle = 0

        if X:
            camera.running = not(camera.running)
            if camera.running:
                AI = True
                camera.observe(update, names='value') 
                print("Start camera/AI running...")
            else:
                camera.unobserve_all()
                AI = False
                print("Stop camera/AI running...")

        if Y or B:  
            save_snapshot_js2v(car.steering,car.throttle)
            print("Get running data!") 

        if S:
            running = False
            print("Stop controlling...")

        if AI == True:
            pass
        elif mode == "exp":
            car.steering = expocurve(round(joystick.get_axis(axis_steer),2) *-1   *0.65)
            car.throttle = expocurve(round(joystick.get_axis(axis_accel),2) *-1   *0.65)
        else:
            car.steering = round(joystick.get_axis(axis_steer),2) *-1   *0.65 #マシン毎調性
            car.throttle = round(joystick.get_axis(axis_accel),2) *-1   *0.65 #マシン毎調性


Joystick Name: Logitech Gamepad F710
Number of Button : 11
Number of Axis : 6
Number of Hats : 1
Breaking!
Breaking!
Breaking!
Breaking!
Breaking!
Breaking!
Breaking!
Breaking!
Breaking!


# カメラの終了処理(必須)
再度、データセット追加や学習は、'02_inreragtive_regression.ipynb'に戻りおこないます。
その際に、下記セルを実行して、カメラの終了処理を忘れずにおこなってください。

In [26]:
import time
camera.running = False
time.sleep(1)
camera.cap.release()